In [ ]:
!pip install rawpy


In [ ]:
import numpy as np
import rawpy
import matplotlib.pyplot as plt
import cv2

In [ ]:
raw_data = np.load('/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_9939.npy')
print(raw_data)
print(raw_data.min())
print(raw_data.max())

# **Exercise One**

In [ ]:
slice_of_raw_data = raw_data[0:10, 0:10]
print(slice_of_raw_data)
plt.imshow(slice_of_raw_data)
plt.title('Slice of Raw Data')
plt.colorbar()
plt.show()

In [ ]:
green_channel_1 = slice_of_raw_data[0::2, 0::2]
green_channel_2 = slice_of_raw_data[1::2, 1::2]
print(green_channel_1)
print(" ")
print(green_channel_2)

In [ ]:
red_channel = slice_of_raw_data[1::2, 0::2]
print(red_channel)

In [ ]:
# Blue channel will be at odd rows and odd columns
blue_channel = slice_of_raw_data[0::2, 1::2]
blue_channel

In [ ]:
green_channel_1 = raw_data[0::2, 0::2]
green_channel_2 = raw_data[1::2, 1::2]
red_channel = raw_data[1::2, 0::2]
blue_channel = raw_data[0::2, 1::2]

# **Exercise Two**

In [ ]:
# File paths
file_3044 = '/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_3044.CR3'
file_3045 = '/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_3045.CR3'
file_3046 = '/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_3046.CR3'
file_3047 = '/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_3047.CR3'
file_3048 = '/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_3048.CR3'
file_3049 = '/content/drive/MyDrive/Colab Notebooks/Exercise 2/IMG_3049.CR3'
# Load the images using rawpy
with rawpy.imread(file_3044) as raw:
    img_3044 = raw.postprocess()
with rawpy.imread(file_3045) as raw:
    img_3045 = raw.postprocess()
with rawpy.imread(file_3046) as raw:
    img_3046 = raw.postprocess()
with rawpy.imread(file_3047) as raw:
    img_3047 = raw.postprocess()
with rawpy.imread(file_3048) as raw:
    img_3048 = raw.postprocess()
with rawpy.imread(file_3049) as raw:
    img_3049 = raw.postprocess()

In [ ]:
# Convert to numpy arrays
array_3044 = np.array(img_3044)
array_3045 = np.array(img_3045)
array_3046 = np.array(img_3046)
array_3047 = np.array(img_3047)
array_3048 = np.array(img_3048)
array_3049 = np.array(img_3049)

# Calculate the average values of red, green, and blue pixels for each image
avg_3044 = np.mean(array_3044, axis=(0, 1))
avg_3045 = np.mean(array_3045, axis=(0, 1))
avg_3046 = np.mean(array_3046, axis=(0, 1))
avg_3047 = np.mean(array_3047, axis=(0, 1))
avg_3048 = np.mean(array_3048, axis=(0, 1))
avg_3049 = np.mean(array_3049, axis=(0, 1))
# Exposure times
exposure_times = np.array([1/10, 1/20, 1/40, 1/80, 1/160, 1/320])

# Plotting the averages against exposure times
plt.figure(figsize=(10, 6))

# Red channel
plt.plot(exposure_times,[avg_3044[0], avg_3045[0],avg_3046[0],avg_3047[0],avg_3048[0],avg_3049[0]], 'r-o', label='Red')

# Green channel
plt.plot(exposure_times,[avg_3044[1], avg_3045[1],avg_3046[1],avg_3047[1],avg_3048[1],avg_3049[1]], 'g-o', label='Green')

# Blue channel
plt.plot(exposure_times,[avg_3044[2], avg_3045[2],avg_3046[2],avg_3047[2],avg_3048[2],avg_3049[2]], 'b-o', label='Blue')

plt.title('Average RGB Values vs. Exposure Time')
plt.xlabel('Exposure Time (seconds)')
plt.ylabel('Average Pixel Value')
plt.legend()
plt.grid(True)

plt.show()


# **Exercise Three**

In [ ]:
import cv2
from scipy.signal import convolve2d
import rawpy
import numpy as np
def read_raw_image(file_path):
    with rawpy.imread(file_path) as raw:
        # Extract the raw data
        raw_data = raw.raw_image_visible.copy()
    return raw_data


def create_color_masks(image_shape):
    """
    Create masks for each color channel based on the Bayer pattern.
    Assumes RGGB pattern by default.
    """
    mask_r = np.zeros(image_shape, dtype=np.uint8)
    mask_g = np.zeros(image_shape, dtype=np.uint8)
    mask_b = np.zeros(image_shape, dtype=np.uint8)

    for row in range(image_shape[0]):
        for col in range(image_shape[1]):
            if (row % 2 == 0) and (col % 2 == 0):
                mask_r[row, col] = 1
            elif (row % 2 == 1) and (col % 2 == 1):
                mask_b[row, col] = 1
            else:
                mask_g[row, col] = 1
            print(mask_r)
            print(mask_g)
            print(mask_b)

    return mask_r, mask_g, mask_b


def convert_to_8bit(image, bit_depth=14):
    return (image / (2**bit_depth - 1)) * 255


def demosaic(raw_image):
    """
    Demosaic the image using the given masks and convolution kernel.
    """
    # Create masks for the RGGB Bayer pattern
    r_mask, g_mask, b_mask = create_color_masks(raw_image.shape)

    # Define the demosaicing kernel (you can use a different kernel if needed)
    demosaic_kernel = np.array([[1.0, 1.0, 1.0],
                                [1.0, 1.0, 1.0],
                                [1.0, 1.0, 1.0]])

    r = convolve2d(raw_image * r_mask, demosaic_kernel, mode='same', boundary='symm')
    g = convolve2d(raw_image * g_mask, demosaic_kernel, mode='same', boundary='symm')
    b = convolve2d(raw_image * b_mask, demosaic_kernel, mode='same', boundary='symm')

    # Normalize
    r /= convolve2d(r_mask, demosaic_kernel, mode='same', boundary='symm')
    g /= convolve2d(g_mask, demosaic_kernel, mode='same', boundary='symm')
    b /= convolve2d(b_mask, demosaic_kernel, mode='same', boundary='symm')

    # Convert the demosaiced channels to 8-bit depth
    r_channel_8bit = convert_to_8bit(r)
    g_channel_8bit = convert_to_8bit(g)
    b_channel_8bit = convert_to_8bit(b)

    color_image = np.dstack((r_channel_8bit, g_channel_8bit, b_channel_8bit)).astype(np.uint8)

    return color_image

In [ ]:
raw_image = read_raw_image("exercise_2_data/03/IMG_4782.CR3")
color_image = demosaic(raw_image)

In [ ]:
import matplotlib.pyplot as plt
# Display the resulting color image
plt.imshow(color_image)
plt.title("Demosaiced Color Image")
plt.axis('off')  # Turn off axis labels and ticks
plt.show()

In [ ]:
# Convert the resulting color_image to YUV color space
yuv_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2YUV)

# Apply median filtering to U and V channels
median_filtered_u = cv2.medianBlur(yuv_image[:, :, 1], ksize=5)  # Adjust kernel size as needed
median_filtered_v = cv2.medianBlur(yuv_image[:, :, 2], ksize=5)  # Adjust kernel size as needed

# Create a new YUV image with the filtered U and V channels
filtered_yuv_image = np.stack((yuv_image[:, :, 0], median_filtered_u, median_filtered_v), axis=-1)

# Convert the filtered YUV image back to RGB
filtered_rgb_image = cv2.cvtColor(filtered_yuv_image, cv2.COLOR_YUV2RGB)

# Display the resulting filtered color image
plt.imshow(filtered_rgb_image)
plt.title("Filtered Color Image")
plt.axis('off')
plt.show()


# **Exercise Four**

In [ ]:
filtered_rgb_image.max()

In [ ]:
# Normalize using percentiles first
a = np.percentile(filtered_rgb_image, 0.01)
b = np.percentile(filtered_rgb_image, 99.99)
normalized_image = (filtered_rgb_image - a) / (b - a)
normalized_image[normalized_image < 0] = 0
normalized_image[normalized_image > 1] = 1

gamma = 0.3
# Apply gamma correction on the normalized image
gamma_corrected_image = normalized_image ** gamma

# Convert to 8-bit type for display, scale up to the 8-bit range
gamma_corrected_image = 255 * gamma_corrected_image
gamma_corrected_image[gamma_corrected_image < 0] = 0
gamma_corrected_image[gamma_corrected_image > 255] = 255
gamma_corrected_image = gamma_corrected_image.astype(np.uint8)

# Display the gamma corrected image
plt.imshow(gamma_corrected_image)
plt.title("Gamma Corrected Image")
plt.axis('off') # Hide the axes
plt.show()

# **Exercise Five**

In [ ]:
def white_balance(gamma_corrected_image):

  mi = np.mean(gamma_corrected_image)

  # Apply white balance to each channel by multiplying them by mi / mc
  white_balanced_image = np.zeros_like(gamma_corrected_image, dtype=np.float64)
  for channel in range(3):  # Iterate over RGB channels
      mc = np.mean(gamma_corrected_image[:, :, channel])
      white_balanced_image[:, :, channel] = gamma_corrected_image[:, :, channel] * (mi / mc)

  # Clip values to a maximum of 255
  white_balanced_image[white_balanced_image > 255] = 255

  # Convert to uint8 type for display
  white_balanced_image = white_balanced_image.astype(np.uint8)

  return white_balanced_image

white_balanced_image = white_balance(gamma_corrected_image)

# Display the white balanced image
plt.imshow(white_balanced_image)
plt.title("White Balanced Image (Gray World)")
plt.axis('off')
plt.show()

In [ ]:
# Create a figure with 1 row and 3 columns for the three images
plt.figure(figsize=(15, 5))

# Plot the filtered_rgb_image on the left
plt.subplot(1, 3, 1)
plt.imshow(filtered_rgb_image)
plt.title("Filtered Color Image")
plt.axis('off')

# Plot the gamma_corrected_image in the middle
plt.subplot(1, 3, 2)
plt.imshow(gamma_corrected_image)
plt.title("Gamma Corrected and Normalized Image")
plt.axis('off')

# Plot the white_balanced_image on the right
plt.subplot(1, 3, 3)
plt.imshow(white_balanced_image)
plt.title("White Balanced Image (Gray World)")
plt.axis('off')

plt.tight_layout()  # Optional for better layout
plt.show()

# ****PART 2****
# Exercise Six

In [ ]:
def simple_hdr_combination(images, exposure_times):
    # Assume the longest exposure is the base (h)
    h = images[0] / exposure_times[0]
    for i, img in enumerate(images[1:]):
        # Multiply by the relative exposure time difference
        exp_diff = exposure_times[i-1] - exposure_times[i]
        img_exp_diff = img * exp_diff
        # Replace values in h with values from img_scaled where img_scaled is greater
        threshold = 0.8 * h.max()
        h = np.where(img_exp_diff > threshold, img_exp_diff, h)
    return h

In [ ]:
def gamma_correction(image, gamma):
    # Apply gamma correction
    return np.power(image, gamma)


In [ ]:
def weighted_hdr_combination(images, exposure_times):
    # Calculate the weights for each image
    weights = [1.0 / t for t in exposure_times]

    # Normalize images based on exposure time and apply weights
    normalized_images = [img * w for img, w in zip(images, weights)]

    # Initialize HDR image with zeros
    hdr_image = np.zeros_like(normalized_images[0])

    # Calculate the weighted sum of normalized images
    for norm_img in normalized_images:
        hdr_image += norm_img

    # Normalize the HDR image to bring the pixel values between 0 and 1
    hdr_image /= np.sum(weights)
        # Apply tone mapping
    tone_mapped_image = np.log(1 + hdr_image) / np.log(2)  # Logarithmic tone mapping

    # Normalize the tone-mapped image to bring the pixel values between 0 and 1
    tone_mapped_image /= np.max(tone_mapped_image)
    gamma_corrected_image = gamma_correction(tone_mapped_image, gamma=0.03)
    # Apply white balance
    final_image = white_balance(gamma_corrected_image)

    return final_image


In [ ]:
file_paths = ['/content/drive/MyDrive/Colab Notebooks/Exercise 6/00.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/01.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/02.CR3',
              '/content/drive/MyDrive/Colab Notebooks/Exercise 6/03.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/04.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/05.CR3',
              '/content/drive/MyDrive/Colab Notebooks/Exercise 6/06.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/07.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/08.CR3',
              '/content/drive/MyDrive/Colab Notebooks/Exercise 6/09.CR3', '/content/drive/MyDrive/Colab Notebooks/Exercise 6/10.CR3']

exposure_times = [1, 1/2, 1/4, 1/8, 1/16, 1/32, 1/64, 1/128, 1/256, 1/512, 1/1024, 1/2048]

# Read and process images
images = [read_raw_image(file_path) for file_path in file_paths]


In [ ]:
images[10].ah

In [ ]:
# Merge to HDR
hdr_image = weighted_hdr_combination(images, exposure_times)

print(hdr_image.dtype)
print(hdr_image.max())
plt.imshow(hdr_image, cmap='gray')
plt.show()

In [ ]:
hdr_image.shape

In [ ]:
white_balanced_image = white_balance(hdr_image)
print(white_balanced_image.dtype)
print(white_balanced_image.shape)
print(white_balanced_image.max())
plt.imshow(white_balanced_image)
plt.show()

In [ ]:
def reduce_dynamic_range(image):
    # Apply logarithmic compression
    image_log = np.log1p(image)
    # Scale to [0, 255]
    image_norm = (image_log - image_log.min()) / (image_log.max() - image_log.min())
    image_8bit = (image_norm * 255).astype(np.uint8)
    return image_8bit

In [ ]:
final_image = reduce_dynamic_range(hdr_image)
print(final_image.dtype)
print(final_image.shape)
print(final_image.max())
plt.imshow(final_image)
plt.show()

In [ ]:
from scipy.signal import convolve2d
demosaiced_image = demosaic(final_image)
print(demosaiced_image.dtype)
print(demosaiced_image.shape)
print(demosaiced_image.max())
plt.imshow(demosaiced_image)
plt.show()

In [ ]:
# Save the image
imageio.imwrite('hdr_image.png', final_image)